In [1]:
import warnings
from datetime import datetime, timedelta
from breeze_connect import BreezeConnect
from snapi_py_client.snapi_bridge import StocknoteAPIPythonBridge
import pandas as pd
import json
import numpy as np
warnings.filterwarnings("ignore")
from snapi_py_client.snapi_bridge import StocknoteAPIPythonBridge
import time

##ICICI LOGIN 
api_key = "---------------------------"
sec_key = "---------------------------"
api_session = '15827182' #ICICI -URL: https://api.icicidirect.com/apiuser/login?api_key=XYZ

# Initialize SDK
breeze = BreezeConnect(api_key=api_key)
breeze.generate_session(api_secret=sec_key,
                        session_token=api_session)


today = datetime.today().strftime('%Y-%m-%d')

# Banifty daily data

downloading banknifty daily data

In [2]:
from_date = "2022-07-17"
to_date = "2023-07-17"

In [3]:
atm = pd.DataFrame({'datetime':[], 'open':[], 'high':[], 'low':[], 'close':[],'volume':[],'open_interest':[]})
atm.to_csv("atm_ce.csv")
atm.to_csv("atm_pe.csv")

In [4]:

bn = breeze.get_historical_data(interval="1day",
                            from_date= from_date,
                            to_date= to_date,
                            stock_code="CNXBAN",
                            exchange_code="NSE",
                            product_type="cash")


In [6]:

bn = pd.DataFrame(bn['Success'])
bn["datetime"] = pd.to_datetime(bn["datetime"])
bn.set_index("datetime", inplace=True)
date_array = bn.index
date_array = date_array.strftime("%Y-%m-%d")
date_array = bn.index
bn['day'] = date_array.strftime("%a")

def next_thursday(date):
    next_thu = date + timedelta((3 - date.weekday()) % 7)
    return next_thu

bn['expiry_date'] = bn.index.map(lambda x: next_thursday(x.date()))
bn['open'] = (bn['open'].astype(float)).astype(int)

bn.to_csv("gdc.csv")


In [8]:
no_data = pd.DataFrame({'date': [], 'strike': [], 'expiry': []})

In [9]:
# bn['strike_price'] = ((bn['open']/100).astype(int)*100)+100
# bn['expiry_date'] = bn['expiry_date'].strftime('%Y-%m-%d')

### downloading option data 

atm


In [10]:
# atm = pd.DataFrame({'datetime':[], 'open':[], 'high':[], 'low':[], 'close':[],'volume':[],'open_interest':[]})
# atm.to_csv("atm_ce.csv")
# atm.to_csv("atm_pe.csv")

profit = pd.DataFrame(columns=['datetime', 'profit'])
atm_diff=100
bn['strike_price'] = ((bn['open']/100).astype(int)*100)+atm_diff
for i in range(0,100):
    date = str(bn.index[i].strftime('%Y-%m-%d'))
    expiry = str(bn.expiry_date[i].strftime('%Y-%m-%d'))
    strike = bn['strike_price'][i]
    try:
        call = pd.DataFrame(breeze.get_historical_data(interval="1minute",
                                        from_date= date,
                                        to_date= date,
                                        stock_code="CNXBAN",
                                        exchange_code="NFO",
                                        product_type="options",
                                        expiry_date=expiry,
                                        right="call",
                                        strike_price=str(int(int(float(bn.open[i]))/100)*100))['Success'])
        call["datetime"] = pd.to_datetime(call["datetime"])
        call = call[['datetime','open','high','low','close','volume','open_interest']]
        call_main = pd.read_csv("atm_ce.csv")
        call_main.drop('Unnamed: 0', axis=1, inplace=True)
        df = pd.concat([call_main, call], ignore_index=True)
        df.to_csv("atm_ce.csv")
        

        put = pd.DataFrame(breeze.get_historical_data(interval="1minute",
                                        from_date= date,
                                        to_date= date,
                                        stock_code="CNXBAN",
                                        exchange_code="NFO",
                                        product_type="options",
                                        expiry_date=expiry,
                                        right="put",
                                        strike_price=str(int(int(float(bn.open[i]))/100)*100))['Success'])
        put["datetime"] = pd.to_datetime(put["datetime"])
        put = put[['datetime','open','high','low','close','volume','open_interest']]
        put_main = pd.read_csv("atm_pe.csv")
        put_main.drop('Unnamed: 0', axis=1, inplace=True)
        df = pd.concat([put_main, call], ignore_index=True)
        df.to_csv("atm_pe.csv")
    except Exception as e:
        print(e)
        time.sleep(2)
        try:
            # Get the expiry date as a datetime object
            expiry = datetime.strptime(bn.expiry_date[i], '%Y-%m-%d')
            call = pd.DataFrame(breeze.get_historical_data(interval="1minute",
                                            from_date= date,
                                            to_date= date,
                                            stock_code="CNXBAN",
                                            exchange_code="NFO",
                                            product_type="options",
                                            expiry_date=expiry,
                                            right="call",
                                            strike_price=str(int(int(float(bn.open[i]))/100)*100))['Success'])
            call["datetime"] = pd.to_datetime(call["datetime"])
            
            call = call[['datetime','open','high','low','close','volume','open_interest']]
            call_main = pd.read_csv("atm_ce.csv")
            call_main.drop('Unnamed: 0', axis=1, inplace=True)
            df = pd.concat([call_main, call], ignore_index=True)
            df.to_csv("atm_ce.csv")
            
            put = pd.DataFrame(breeze.get_historical_data(interval="1minute",
                                            from_date= date,
                                            to_date= date,
                                            stock_code="CNXBAN",
                                            exchange_code="NFO",
                                            product_type="options",
                                            expiry_date=expiry,
                                            right="put",
                                            strike_price=str(int(int(float(bn.open[i]))/100)*100))['Success'])
            put["datetime"] = pd.to_datetime(put["datetime"])
            put = put[['datetime','open','high','low','close','volume','open_interest']]
            put_main = pd.read_csv("atm_pe.csv")
            put_main.drop('Unnamed: 0', axis=1, inplace=True)
            df = pd.concat([put_main, call], ignore_index=True)
            df.to_csv("atm_pe.csv")
        except Exception as e:
            print(e)
            time.sleep(2)
            try: 
                # Get the expiry date as a datetime object
                expiry = datetime.strptime(bn.expiry_date[i], '%Y-%m-%d')

                # Subtract 1 day from the expiry date
                expiry = expiry - datetime.timedelta(days=1)
                expiry = expiry.strftime('%Y-%m-%d')
                call = pd.DataFrame(breeze.get_historical_data(interval="1minute",
                                                from_date= date,
                                                to_date= date,
                                                stock_code="CNXBAN",
                                                exchange_code="NFO",
                                                product_type="options",
                                                expiry_date=expiry,
                                                right="call",
                                                strike_price=str(int(int(float(bn.open[i]))/100)*100))['Success'])
                call["datetime"] = pd.to_datetime(call["datetime"])
                
                call = call[['datetime','open','high','low','close','volume','open_interest']]
                call_main = pd.read_csv("atm_ce.csv")
                call_main.drop('Unnamed: 0', axis=1, inplace=True)
                df = pd.concat([call_main, call], ignore_index=True)
                df.to_csv("atm_ce.csv")
                
                put = pd.DataFrame(breeze.get_historical_data(interval="1minute",
                                                from_date= date,
                                                to_date= date,
                                                stock_code="CNXBAN",
                                                exchange_code="NFO",
                                                product_type="options",
                                                expiry_date=expiry,
                                                right="put",
                                                strike_price=str(int(int(float(bn.open[i]))/100)*100))['Success'])
                put["datetime"] = pd.to_datetime(put["datetime"])
            
                put = put[['datetime','open','high','low','close','volume','open_interest']]
                put_main = pd.read_csv("atm_pe.csv")
                put_main.drop('Unnamed: 0', axis=1, inplace=True)
                df = pd.concat([put_main, call], ignore_index=True)
                df.to_csv("atm_pe.csv")

            except Exception as e:
                print(e)
                try: 
                    # Get the expiry date as a datetime object
                    expiry = datetime.strptime(bn.expiry_date[i], '%Y-%m-%d')

                    # Subtract 1 day from the expiry date
                    expiry = expiry - datetime.timedelta(days=2)
                    expiry = expiry.strftime('%Y-%m-%d')
                    call = pd.DataFrame(breeze.get_historical_data(interval="1minute",
                                                    from_date= date,
                                                    to_date= date,
                                                    stock_code="CNXBAN",
                                                    exchange_code="NFO",
                                                    product_type="options",
                                                    expiry_date=expiry,
                                                    right="call",
                                                    strike_price=str(int(int(float(bn.open[i]))/100)*100))['Success'])
                    call["datetime"] = pd.to_datetime(call["datetime"])
                    
                    call = call[['datetime','open','high','low','close','volume','open_interest']]
                    call_main = pd.read_csv("atm_ce.csv")
                    call_main.drop('Unnamed: 0', axis=1, inplace=True)
                    df = pd.concat([call_main, call], ignore_index=True)
                    df.to_csv("atm_ce.csv")
                    
                    put = pd.DataFrame(breeze.get_historical_data(interval="1minute",
                                                    from_date= date,
                                                    to_date= date,
                                                    stock_code="CNXBAN",
                                                    exchange_code="NFO",
                                                    product_type="options",
                                                    expiry_date=expiry,
                                                    right="put",
                                                    strike_price=str(int(int(float(bn.open[i]))/100)*100))['Success'])
                    put["datetime"] = pd.to_datetime(put["datetime"])
                
                    put = put[['datetime','open','high','low','close','volume','open_interest']]
                    put_main = pd.read_csv("atm_pe.csv")
                    put_main.drop('Unnamed: 0', axis=1, inplace=True)
                    df = pd.concat([put_main, call], ignore_index=True)
                    df.to_csv("atm_pe.csv")
                except:
                    no_data = no_data.append({'date': date, 'strike': int(strike), 'expiry': expiry}, ignore_index=True)

'Success'
strptime() argument 1 must be str, not datetime.date
strptime() argument 1 must be str, not datetime.date
'Success'
strptime() argument 1 must be str, not datetime.date
strptime() argument 1 must be str, not datetime.date
'Success'
strptime() argument 1 must be str, not datetime.date
strptime() argument 1 must be str, not datetime.date


In [11]:
#check if data is working

# dfv = pd.read_csv("atm_pe.csv")
# dfv

In [12]:
no_data #data not found after 4 trys expiry -2 days was tried max

,date,strike,expiry
0,2022-09-28,38100.0,2022-09-29
1,2022-09-29,38100.0,2022-09-29
2,2022-09-30,37700.0,2022-10-06


trying to download no_data's data with 2 more try's